<a href="https://colab.research.google.com/github/dtype2100/TIL/blob/master/Naver_news_report_talk_Top2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install top2vec

In [ ]:
pip install top2vec[sentence_encoders]

In [ ]:
pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
  Using cached JPype1-1.3.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (448 kB)


In [ ]:
pip install pdfminer.six

     |████████████████████████████████| 5.6 MB 4.7 MB/s 
     |████████████████████████████████| 3.6 MB 43.2 MB/s 


In [ ]:
import pprint
import pandas as pd
import tensorflow_hub as hub


from pdfminer.high_level import extract_pages
from pdfminer.high_level import extract_text
from top2vec import Top2Vec
from konlpy.tag import Okt

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
file_text = repr(file_path)
df5 = file_text.replace('\\n', '')

In [ ]:
df6.rename(columns={'title':'제목'}, inplace=True)
df7.rename(columns={'title':'제목'}, inplace=True)

In [ ]:
df = pd.concat([df1, df2, df3, df4, df6, df7]) # 파일 합치기 (종합할 데이터 없으면 각주처리)

In [ ]:
#df = pd.concat([df1, df2, df3, df4]) # 파일 합치기 (종합할 데이터 없으면 각주처리)
title_df = df[df['제목'].str.contains('네이버')] # 해당 키워드를 포함한 제목 추출 (불러온 데이터에 따라 키워드 변경)
title_df.drop_duplicates(subset=['제목'], inplace=True) # 중복 제거
new_df = pd.DataFrame({'일자' : pd.to_datetime(title_df['일자'], format='%Y%m%d'), '제목' : title_df['제목'], 'URL' : title_df['URL']}) # datetime 데이터프레임 생성
mask = (new_df['일자'] >= '2020-03-31') & (new_df['일자'] <= '2020-08-31') # 날짜 변경
period_df = new_df.loc[mask] # 날짜 인덱싱

str_df = period_df['제목'] + df5

lis_df = list(str_df) # 데이터 타입 list로 변경

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
str_df

1080     [네이버 테크인사이드] ㊾ 농업도 '라인웍스'로 디지털전환... "이제 카톡으로 업...
1081     데스커, 네이버 브랜드데이 ‘게이밍 데스크’ 첫선"2020. 7. 8 인터넷/게임 ...
1082     데스커, '네이버 브랜드데이' 진행 게이밍 데스크 최초 공개"2020. 7. 8 인...
1083     블록체인 사업 채비 마친 '네이버 라인' 다음 행보에 쏠린 눈"2020. 7. 8 ...
1084     '온라인 장보기' 내놓은 네이버, 쿠팡 마켓컬리 넘어설까"2020. 7. 8 인터넷...
                               ...                        
12866    네이버웹툰, 유료콘텐츠 하루 거래액 30억 돌파 업계 최초"2020. 7. 8 인터...
13697    "다음주까지 못 기다려" 네이버웹툰, 1일 거래액 30억 돌파"2020. 7. 8 ...
15726    네이버웹툰 하루 거래액 30억원 돌파 올해 8000억원 목표"2020. 7. 8 인...
15797    ‘승승장구’ 네이버웹툰, 글로벌 하루 거래액 30억 돌파 연간 1조원 가능할까"20...
17740    페이스북 신무기는 ‘리브라 페이’? 네이버 카카오와 경쟁 예고"2020. 7. 8 ...
Name: 제목, Length: 1013, dtype: object

In [ ]:
# df = pd.concat([df1, df2, df3, df4]) # 파일 합치기 (종합할 데이터 없으면 각주처리)
# title_df = df[df['제목'].str.contains('네이버')] # 해당 키워드를 포함한 제목 추출 (불러온 데이터에 따라 키워드 변경)
# title_df.drop_duplicates(subset=['제목'], inplace=True) # 중복 제거
# new_df = pd.DataFrame({'일자' : pd.to_datetime(title_df['일자'], format='%Y%m%d'), '제목' : title_df['제목'], 'URL' : title_df['URL']}) # datetime 데이터프레임 생성
# mask = (new_df['일자'] >= '2020-03-31') & (new_df['일자'] <= '2020-08-31') # 날짜 변경
# period_df = new_df.loc[mask] # 날짜 인덱싱

# okt = Okt()
# stop_words = '네이버와 카카오'
# stop_words = set(stop_words.split(' '))
# word_tokens = okt.morphs(str(period_df['제목']))
# li_df = [word for word in word_tokens if not word in stop_words]

# lis_df = list(li_df)

# 모델 훈련

In [ ]:
embed = hub.load("https://tfhub.dev/google/nnlm-ko-dim50/2")
model = Top2Vec(documents=lis_df, speed="learn", workers=8, embedding_model=embed)

2022-05-16 17:24:07,705 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
2022-05-16 17:30:46,221 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2022-05-16 17:30:57,526 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-05-16 17:31:

# Get Number of Topics

In [ ]:
model.get_num_topics()

1

# Get Topics Sizes

In [ ]:
topic_sizes, topic_nums = model.get_topic_sizes()

# Get Topics

In [ ]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["네이버"], num_topics=1)

In [ ]:
topic_words

[array(['수익', '실적', '매출', '이익률', '매출액', '트래픽', '수익성', '손익', '이익', 'cash',
        '순이익', '마진', '수익률', '밸류에이션', '매출원가', '안정성', '컨센서스', '가격', '고정비',
        '컨텐츠', '마케팅비', '추정치', '데이터', '판매량', '사용량', '성장성', 'valuation',
        '포트폴리오', '괴리율', '어플리케이션', '순매출', '지배력', '자산', '신사업', '증감', '컨버젼',
        '거래', '판매', '거래액', '경쟁력', '외형', '콘텐츠', '수요', '모멘텀', '투자', '역량',
        '가치', '투자액', '조달', '지분율'], dtype='<U9')]

# Generate Word Clouds

In [ ]:
# topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["네이버"], num_topics=1)
# for topic in topic_nums:
#     model.generate_topic_wordcloud(topic)

# Search Documents by Topic

In [ ]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=1, num_docs=10)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

ValueError: ignored

#Semantic Search Documents by Keywords

In [ ]:
documents, document_scores, document_ids = model.search_documents_by_keywords(keywords=["네이버"], num_docs=10)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

#Similar Keywords

In [ ]:
words, word_scores = model.similar_words(keywords=["네이버"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

포털 0.8457718528943259
유튜브 0.7709171159829908
사이트 0.7295019292113227
야후 0.7166914242224498
구글 0.6975502301906484
쿠팡 0.6888635452903606
인터넷 0.6856421922500521
검색 0.6795214642597057
옥션 0.669601001074344
온라인 0.6578929675391725
메신저 0.6222371298666968
안드로이드 0.6094683291016594
웹툰 0.606801447646618
게임빌 0.6039145517125131
컴투스 0.6005436266182747
넷마블 0.5943452839471339
모바일 0.5888586798143296
방문자 0.5886071534443067
넥슨 0.5852483392140049
다운로드 0.5784230559362901
